In [19]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

def train_xor(x, y, epochs=100000, lr=0.05):
    # Khởi tạo trọng số nhỏ
    w1 = np.random.uniform(-1, 1, (2, 2))
    b1 = np.random.uniform(-1, 1, (1, 2))
    w2 = np.random.uniform(-1, 1, (2, 1))
    b2 = np.random.uniform(-1, 1)

    for epoch in range(epochs):
        # Forward
        z1 = np.dot(x, w1) + b1     # (4,2)
        a1 = sigmoid(z1)

        z2 = np.dot(a1, w2) + b2    # (4,1)
        a2 = sigmoid(z2)

        # Loss
        loss = -np.mean(y * np.log(a2) + (1 - y) * np.log(1 - a2))

        # Backprop
        dz2 = a2 - y
        dw2 = np.dot(a1.T, dz2) / 4
        db2 = np.sum(dz2, axis=0, keepdims=True) / 4

        dz1 = np.dot(dz2, w2.T) * sigmoid_derivative(z1)
        dw1 = np.dot(x.T, dz1) / 4
        db1 = np.sum(dz1, axis=0, keepdims=True) / 4

        # Cập nhật
        w2 -= lr * dw2
        b2 -= lr * db2
        w1 -= lr * dw1
        b1 -= lr * db1

    return x, y, a2, np.round(a2), loss


x = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
])
y = np.array([[0], [1], [1], [0]])

# Huấn luyện
x, y, output, prediction, loss = train_xor(x, y)

# Kết quả
for i in range(4):
    print(f"Input: {x[i]}, Expected: {y[i][0]}, Predicted: {prediction[i][0]}")

print("Loss:", loss)


Input: [0 0], Expected: 0, Predicted: 0.0
Input: [0 1], Expected: 1, Predicted: 1.0
Input: [1 0], Expected: 1, Predicted: 1.0
Input: [1 1], Expected: 0, Predicted: 0.0
Loss: 0.0023006687210754516
